# Spacy relationship extraction

Use the pip_spacy Conda environment

* Claucy
* Concepcy

In [2]:
import spacy
import psycopg2
import pandas as pd
from spacy.tokens import Span
from spacy import displacy


In [3]:
from itables import init_notebook_mode, show
import re
from importlib import reload

In [4]:
import coreferee

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import postgresql_functions as pgf
import settings as stt

In [6]:
# reload(pgf)

In [7]:
### connect to the local database
conn = psycopg2.connect(host="localhost", port = 5432, database="espace_intellectuel", 
                        user="postgres", password=stt.dbw)
#conn

In [74]:
q1 = """
select pk_mathshistory, "name", url, dates, length(biography) as eff, coreferenced_txt 
from mathshistory.mathshistory m 
where pk_mathshistory in (103, 117, 133, 159, 186);
"""

In [75]:
result = pgf.sql_explore(q1, conn)
# print(f'Lines count: {len(result[0])}, errors count: {len(result[1])}, \nFirst lines: {result[0][:5]}')

In [76]:
textes = pd.DataFrame(result[0])
textes.columns = ['id', 'name', 'url', 'dates', 'length_bio', 'texte']

In [77]:
textes.head()

,id,name,url,dates,length_bio,texte
0,103,Christopher Clavius,https://mathshistory.st-andrews.ac.uk/Biograph...,1538-1612,10616,Christopher Clavius was born in a German regio...
1,117,Michael Mästlin,https://mathshistory.st-andrews.ac.uk/Biograph...,1550-1631,10122,Michael Mästlin was born in Göppingen which wa...
2,133,Giuseppe Biancani,https://mathshistory.st-andrews.ac.uk/Biograph...,1566-1624,10035,Giuseppe Biancani's name also appears in name ...
3,159,Wilhelm Schickard,https://mathshistory.st-andrews.ac.uk/Biograph...,1592-1635,10272,Wilhelm Schickard's name is sometimes written ...
4,186,Johannes Hevelius,https://mathshistory.st-andrews.ac.uk/Biograph...,1611-1687,10856,The first problem that we have to address is t...


In [78]:
### Choose one document
txt = textes.iloc[1].texte 
print(txt)

Michael Mästlin was born in Göppingen which was a village about 50 km east of Tübingen. Mästlin father, Jakob Mästlin, and Mästlin mother, Dorothea Simon, were both devout Lutherans and Michael was brought up in that faith and remained strongly committed to father throughout Mästlin life. Mästlin was the middle child of the family, having an older sister and a younger brother. Mästlin attended the monastic school in Königsbronn then, after Mästlin studies there, entered Tübingen University in 1568. [3]:-
As was the case with many young scholars including Kepler, Mästlin most famous student, [Mästlin ] did Mästlin undergraduate studies at a preparatory school and came to the university to take Mästlin final exams and pick up Mästlin baccalaureate degree.At Tübingen University Mästlin studied mathematics and astronomy for a Master's degree under Philipp Apian who was Peter Apian's son. In 1570, while a student, Mästlin purchased a copy of Copernicus's De revolutionibus from the widow of 

In [15]:
#txt = "We have quoted above from Biancani concerning his high regard for Galileo. However, he did not always agree with Galileo's views. The first disagreement came in 1611 and concerned the mountains on the moon. Galileo had observed the surface of the moon through a telescope in 1609 and had used certain mathematical techniques to prove that there were lunar mountains. His claim appeared in Sidereus Nuncius published in May 1610. In May 1611 a group of scientists, mostly Jesuits, was brought together by cardinal Ferdinando Gonzaga in Mantua to discuss Galileo's claims. One of the major points discussed was Galileo's proof that there were mountains on the moon, and the report from the group came down firmly in favour of the traditional belief that the moon was perfectly smooth. Galileo suspected that Biancani was the author of the report and letters were exchanged in which Biancani dissociated himself from any insult towards Galileo saying that he was sorry if he had been offended but, nevertheless, pointing out that he did believe that the moon was perfectly smooth. He also disagreed with Galileo in 1613 when a dispute broke out between Galileo and Christoph Scheiner over sunspots. Galileo unfairly accused Scheiner of plagiarism but, although Scheiner's discovery of sunspots was certainly independent of any work by Galileo, his explanation was quite wrong. Biancani, however, defended his fellow Jesuit Scheiner."

In [82]:
nlp = spacy.load('en_core_web_lg')

In [40]:
### https://spacy.io/universe/project/coreferee
# nlp.add_pipe('coreferee')

In [80]:
# nlp.add_pipe('opentapioca')

In [83]:
nlp.add_pipe("entityLinker", last=True)

In [84]:
doc = nlp(txt)

In [100]:
for span in doc.ents:
    print((span.text, span.kb_id_, span.start, span.label_, span._.description, span._.score))

('Michael Mästlin', '', 0, 'PERSON', None, None)
('Göppingen', '', 5, 'PERSON', None, None)
('about 50 km', '', 10, 'QUANTITY', None, None)
('Tübingen', '', 15, 'ORG', None, None)
('Mästlin', '', 17, 'PERSON', None, None)
('Jakob Mästlin', '', 20, 'PERSON', None, None)
('Mästlin', '', 24, 'PERSON', None, None)
('Dorothea Simon', '', 27, 'PERSON', None, None)
('Lutherans', '', 33, 'NORP', None, None)
('Michael', '', 35, 'PERSON', None, None)
('Mästlin', '', 49, 'PERSON', None, None)
('Mästlin', '', 52, 'PERSON', None, None)
('Mästlin', '', 70, 'PERSON', None, None)
('Königsbronn', '', 76, 'GPE', None, None)
('Mästlin', '', 80, 'PERSON', None, None)
('Tübingen University', '', 85, 'ORG', None, None)
('1568', '', 88, 'DATE', None, None)
('3]:-', '', 91, 'CARDINAL', None, None)
('Kepler', '', 102, 'PERSON', None, None)
('Mästlin', '', 104, 'PERSON', None, None)
('Mästlin', '', 110, 'PERSON', None, None)
('Mästlin', '', 113, 'PERSON', None, None)
('Mästlin', '', 127, 'PERSON', None, None)
(

In [ ]:
for e in doc._.linkedEntities:
    print(e.label, e.url, e.span, e.description)

In [113]:
sents = [s for s in doc.sents]

In [115]:
st = sents[5]; st

At Tübingen University Mästlin studied mathematics and astronomy for a Master's degree under Philipp Apian who was Peter Apian's son.

## Relation extraction

In [66]:
### https://towardsdatascience.com/nlp-with-python-knowledge-graph-12b93146a458

def extract_entities(doc):
    a, b, prev_dep, prev_txt, prefix, modifier = "", "", "", "", "", ""
    for token in doc:
        if token.dep_ != "punct":
            ## prexif --> prev_compound + compound
            if token.dep_ == "compound":
                prefix = prev_txt +" "+ token.text if prev_dep == "compound" else token.text
            
            ## modifier --> prev_compound + %mod
            if token.dep_.endswith("mod") == True:
                modifier = prev_txt +" "+ token.text if prev_dep == "compound" else token.text
            
            ## subject --> modifier + prefix + %subj
            if token.dep_.find("subj") == True:
                a = modifier +" "+ prefix + " "+ token.text
                prefix, modifier, prev_dep, prev_txt = "", "", "", ""
            
            ## if object --> modifier + prefix + %obj
            if token.dep_.find("obj") == True:
                b = modifier +" "+ prefix +" "+ token.text
            
            prev_dep, prev_txt = token.dep_, token.text
    
    # clean
    a = " ".join([i for i in a.split()])
    b = " ".join([i for i in b.split()])
    return (a.strip(), b.strip())


# The relation extraction requires the rule-based matching tool, 
# an improved version of regular expressions on raw text.
def extract_relation(doc, nlp):
    matcher = spacy.matcher.Matcher(nlp.vocab)
    p1 = [{'DEP':'ROOT'}, 
          {'DEP':'prep', 'OP':"?"},
          {'DEP':'agent', 'OP':"?"},
          {'POS':'ADJ', 'OP':"?"}] 
    matcher.add(key="matching_1", patterns=[p1]) 
    matches = matcher(doc)
    k = len(matches) - 1
    span = doc[matches[k][1]:matches[k][2]] 
    return span.text

In [113]:
sents = [s for s in doc.sents]

In [122]:
st = sents[5]

In [123]:
info = []
for token in st:
    info.append((token.text, token.lemma_, token.ent_kb_id_, token.ent_type_, token.pos_, token.tag_, token.dep_,
                token.shape_, token.is_alpha, token.is_stop))
tx_df  = pd.DataFrame(info, columns = ["text", "lemma_", "kb_id", "ent_type", "pos_", "tag_", "dep_","shape_", "is_alpha", "is_stop"])

In [124]:
### https://github.com/mwouts/itables/blob/main/docs/advanced_parameters.md
show(tx_df, classes="display", scrollY="200px", scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")

In [125]:
extract_entities(st)

('Philipp who', 'Philipp Apian')

In [126]:
extract_relation(st, nlp)

'studied'